In [56]:
import os
from dotenv import load_dotenv
load_dotenv()
# Load environment variables
uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

In [57]:
from langchain_neo4j import Neo4jGraph
from neo4j import GraphDatabase

# Initialize the Neo4j driver, which is used to connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password))

In [58]:
## Dataset Movie
# write the cypher query to load the movie dataset into Neo4j
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [59]:
# Run in a session
with driver.session() as session:
    session.run(moview_query)

driver.close()

In [60]:
def run_query(tx, query):
    return list(tx.run(query))

In [61]:
with driver.session() as session:
    query = "MATCH (m:Movie) RETURN m.title AS title, m.imdbRating AS rating LIMIT 5"
    result = session.read_transaction(run_query, query)
    for record in result:
        print(record["title"], "-", record["rating"])

C:\Users\lalra\AppData\Local\Temp\ipykernel_35756\674865814.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\lalra\AppData\Local\Temp\ipykernel_35756\674865814.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(run_query, query)


Toy Story - 8.3
Jumanji - 6.9
Grumpier Old Men - 6.6
Waiting to Exhale - 5.6
Father of the Bride Part II - 5.9


In [62]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    refresh_schema=True
)

ValueError: Query object is only supported for session.run

In [ ]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000257D133E7B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000257D133F8F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain.chains import GraphCypherQAChain
# Create the Cypher QA chain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

ValidationError: 1 validation error for GraphCypherQAChain
graph
  Input should be an instance of GraphStore [type=is_instance_of, input_value=<langchain_neo4j.graphs.n...t at 0x00000257D0FECCE0>, input_type=Neo4jGraph]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of

In [ ]:
# Ask a question
query = "Who directed the movie Inception and what is its IMDb rating?"
result = chain.run(query)
print("Answer:", result['result'])